**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/mario/WoConDock/software'
protein_file = '/home/mario/WoConDock/wocondock_performance_ace_test/receptor_protoss_prepared.pdb'
ref_file = '/home/mario/WoConDock/wocondock_performance_ace_test/crystal_ligand_protoss.sdf'
docking_library = '/home/mario/WoConDock/wocondock_performance_ace_test/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[09:16:50] Initializing Normalizer


The working directory has been set to: /home/mario/WoConDock/wocondock_performance_ace_test
The folder: /home/mario/WoConDock/wocondock_performance_ace_test/temp already exists


In [ ]:
pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver')

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted_futures(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
fetch_poses_splitted(w_dir, n_poses, split_files_folder=w_dir+'/temp/split_final_library')

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

In [ ]:
for metric in clustering_metrics:
    cluster_futures(f'{metric}', 'KMedoids', w_dir, protein_file, all_poses)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [3]:
for metric in clustering_metrics:
    rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 0)


The folder: /home/mario/WoConDock/wocondock_performance_ace_test/temp/rescoring_RMSD_clustered was created
The folder: /home/mario/WoConDock/wocondock_performance_ace_test/temp/rescoring_RMSD_clustered/gnina_rescoring/ was created
Rescoring with GNINA


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

/tmp/libmolgrid/src/grid_maker.cu:288: no kernel image is available for execution on the deviceterminate called after throwing an instance of 'std::runtime_error'
  what():  CUDA Error: no kernel image is available for execution on the device
*** Aborted at 1676967423 (unix time) try "date -d @1676967423" if you are using GNU date ***
PC: @                0x0 (unknown)
*** SIGABRT (@0x3e8000052ae) received by PID 21166 (TID 0x7ff5f5fff640) from PID 21166; stack trace: ***
    @     0x7ff617e42520 (unknown)
    @     0x7ff617e96a7c pthread_kill
    @     0x7ff617e42476 raise
    @     0x7ff617e287f3 abort
    @     0x7ff6182a2bbe (unknown)
    @     0x7ff6182ae24c (unknown)
    @     0x7ff6182ae2b7 std::terminate()
    @     0x7ff6182ae518 __cxa_throw
    @           0xbcc1f4 libmolgrid::GridMaker::forward<>()
    @           0x661ec8 caffe::MolGridDataLayer<>::set_grid_minfo(

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.0 HEAD:6381355   Built Mar  6 2021.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r /home/mario/WoConDock/wocondock_performance_ace_test/receptor_protoss_prepared.pdb -l /home/mario/WoConDock/wocondock_performance_ace_test/temp/clustering/RMSD_clustered.sdf --autobox_ligand /home/mario/WoConDock/wocondock_performance_ace_test/crystal_ligand_protoss.sdf -o /home/mario/WoConDock/wocondock_performance_ace_test/temp/rescoring_RMSD_clustered/gnina_rescoring/rescored_crossdock_default2018.sdf --cnn crossdock_default2018 --score_only
## Name gauss(o=0,_w=0.5,_c=8) gauss(o=3,_w=2,_c=8) repulsion(o=0,_c=8) hydrophobic(g=0.5,_b=1.5,_c=8) non_dir_h_bond(g=-0.7,_b=0,_c=8) num_tors_div


KeyError: "None of [Index(['Pose ID', 'GNINA_Affinity', 'GNINA_CNN_Score', 'GNINA_CNN_Affinity'], dtype='object')] are in the [columns]"

**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_consensus_methods(w_dir, ['bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS'])

In [ ]:
calculate_EFs(w_dir, docking_library)